In MindSpore there are [two execution modes](https://mindspore.cn/tutorials/en/r1.10/advanced/compute_graph/mode.html#dynamic-and-static-graphs): a static graph mode (GRAPH_MODE) and a dynamic graph mode (PYNATIVE_MODE).

*   In dynamic graph mode, also known as python native mode for the way its execution, the program is executed line by line. During the forward execution process, the backward execution graph is generated DNAMICALLY in accordance with the backward propagation principle.

*   In static graph mode, when the program is built and executed, firstly the computation  graph of the neural network is generated. After that the computation graph is performed (by the means of all operations in it) .

Therefore, when static graph mode is set, the neural network model training and inference performs much faster than in the python native mode. Such a boost is possible due to encapsulated graph optimization and cross-platform running.

Lets appeal to example: 

In [15]:
# Firstly install mindspore 1.9.0
#  The specified version is for CPU only. In order to use version for GPU,
#  type 'mindspore_gpu==1.9.0'
!pip install mindspore==1.9.0
!pip install download

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
from download import download
import numpy as np
import mindspore as ms
from mindspore import nn, Model, Parameter, Tensor
from mindspore.dataset import vision, transforms, MnistDataset
from mindspore.common.initializer import Normal
from mindspore import dtype as mstype

In [2]:
# Download MNIST dataset and define preprocessing procedure
url = "https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/" \
      "notebook/datasets/MNIST_Data.zip"

download(url, "./", kind="zip", replace=True)

def proc_dataset(data_path, batch_size=32):
    mnist_ds = MnistDataset(data_path, shuffle=True)

    # preprocess transformation operations
    image_transforms = [
        vision.Resize(32),
        vision.Rescale(1.0 / 255.0, 0),
        vision.Normalize(mean=(0.1307,), std=(0.3081,)),
        vision.HWC2CHW()
    ]

    label_transform = transforms.TypeCast(mstype.int32)

    mnist_ds = mnist_ds.map(operations=label_transform, input_columns="label")
    mnist_ds = mnist_ds.map(operations=image_transforms, input_columns="image")
    mnist_ds = mnist_ds.batch(batch_size, drop_remainder=True)
    return mnist_ds


file_sizes: 100%|██████████████████████████| 10.8M/10.8M [00:01<00:00, 10.2MB/s]
Extracting zip file...
Successfully downloaded / unzipped to ./


In [3]:
# Define simple network structure.
class LeNet5(nn.Cell):

    def __init__(self, num_class=10, num_channel=1):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(num_channel, 6, 5, pad_mode='valid')
        self.conv2 = nn.Conv2d(6, 16, 5, pad_mode='valid')
        self.relu = nn.ReLU()
        self.max_pool2d = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Dense(16 * 5 * 5, 120, weight_init=Normal(0.02))
        self.fc2 = nn.Dense(120, 84, weight_init=Normal(0.02))
        self.fc3 = nn.Dense(84, num_class, weight_init=Normal(0.02))
        self.counted = Parameter(Tensor(np.zeros((1))), requires_grad=False)

    def construct(self, x):
        if self.counted != Tensor(np.ones((1))):
            print("\nThis string is only printed in the PYNATIVE_MODE\n")
            self.counted = Tensor(np.ones((1)))
        x = self.conv1(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

def create_model():
    model = LeNet5()
    net_loss = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction="mean")
    net_opt = nn.Momentum(model.trainable_params(), learning_rate=0.01, momentum=0.9)
    trainer = Model(model, loss_fn=net_loss, optimizer=net_opt, metrics={"Accuracy": nn.Accuracy()})
    return trainer

In [5]:
# Defining callbacks
from mindspore import LossMonitor
from mindspore import CheckpointConfig, ModelCheckpoint
from mindspore import TimeMonitor

loss_monitor = LossMonitor(1875)
config = CheckpointConfig(save_checkpoint_steps=1875, keep_checkpoint_max=10)
time_monitor = TimeMonitor(1875)
ckpt_callback = ModelCheckpoint(prefix="mnist", directory="./checkpoint", config=config)

Dynamic graphs are executed through explanation, with dynamic syntax affinity and flexible expression. Why may one need use dynamic graph (python native) mode ? The most obvious answer is: for [debug purpose](https://mindspore.cn/tutorials/en/r1.10/advanced/compute_graph/mode.html#mode-selection). In dynamic graph mode, it is possible to use breakpoints and observe intermediate results of network execution ( [inside](https://mindspore.cn/tutorials/en/r1.10/beginner/quick_start.html#building-network) `def construct(self, input)` method, that encapsulates the construction process of the computational graph. 

In [6]:
train_dataset = proc_dataset('MNIST_Data/train')
test_dataset = proc_dataset('MNIST_Data/test')

# Configuring the dynamic graph mode
ms.set_context(mode=ms.PYNATIVE_MODE)

trainer = create_model()
trainer.train(3, train_dataset, callbacks=[ckpt_callback, loss_monitor, time_monitor])
print("\nEvaluation:\n")
trainer.fit(2, train_dataset, test_dataset, callbacks=[loss_monitor])


This string is only printed in the PYNATIVE_MODE

epoch: 1 step: 1875, loss is 0.10879134386777878
Train epoch time: 59068.311 ms, per step time: 31.503 ms
epoch: 2 step: 1875, loss is 0.04291274771094322
Train epoch time: 58152.894 ms, per step time: 31.015 ms
epoch: 3 step: 1875, loss is 0.0006521427421830595
Train epoch time: 54736.380 ms, per step time: 29.193 ms

Evaluation:

epoch: 1 step: 1875, loss is 0.00042648648377507925
Eval result: epoch 1, metrics: {'Accuracy': 0.9897836538461539}
epoch: 2 step: 1875, loss is 0.0011952654458582401
Eval result: epoch 2, metrics: {'Accuracy': 0.989082532051282}


Pay attention to the first string in the cell output. This line only printed in the python native mode. That's because in static graph mode it is not compiled as the operation of the computation graph.

In static graph mode, that is the default mode in Mindspore, no breakpoints can be set. That's because of the way of [graph execution](https://mindspore.cn/tutorials/en/r1.10/advanced/compute_graph/mode.html#execution-principle-of-the-static-graph-mode).

In static graph mode, MindSpore converts the Python source code into an [intermediate representation](https://mindspore.cn/tutorials/experts/en/r1.10/debug/mindir.html#reading-ir), optimizes the IR graph and executes the optimized graph on the hardware device. Static graphs are executed through just in time (JIT) build

In [7]:
train_dataset = proc_dataset('MNIST_Data/train')
test_dataset = proc_dataset('MNIST_Data/test')

# Configuring the static graph mode
ms.set_context(mode=ms.GRAPH_MODE)

trainer = create_model()
trainer.train(3, train_dataset, callbacks=[ckpt_callback, loss_monitor, time_monitor])
print("\nEvaluation:\n")
trainer.fit(2, train_dataset, test_dataset, callbacks=[loss_monitor])

epoch: 1 step: 1875, loss is 0.08928205072879791
Train epoch time: 43178.599 ms, per step time: 23.029 ms
epoch: 2 step: 1875, loss is 0.3162972927093506
Train epoch time: 42310.823 ms, per step time: 22.566 ms
epoch: 3 step: 1875, loss is 0.025710901245474815
Train epoch time: 40802.322 ms, per step time: 21.761 ms

Evaluation:

epoch: 1 step: 1875, loss is 0.07349871098995209
Eval result: epoch 1, metrics: {'Accuracy': 0.9863782051282052}
epoch: 2 step: 1875, loss is 0.04860566556453705
Eval result: epoch 2, metrics: {'Accuracy': 0.9887820512820513}


From now we can see that execution time is twice less for the static graph mode.


MindSpore also supports using [control statements](https://mindspore.cn/tutorials/experts/en/r1.10/network/control_flow.html#process-control-statements) in the static graph mode.
This is especially useful when dealing with conditional flows in the computation of the network. For example when implementing custom picewise defined activation functions

  It is possible to combine dynamic and static graphs in a single runtime. For that purpose `ms_function` modifier might be applied to any method object that needs to be executed using static graphs. For [example](https://mindspore.cn/tutorials/en/r1.10/advanced/compute_graph/combine.html#implementation-principle):


```
class Mul(nn.Cell):
    """Define a class to implement the x self multiplication."""
    @ms_function  # Use ms_function to modify the function. This function is executed in static graph mode.
    def construct(self, x):
        x = x * x
        x = x * x
        return x
ms.set_context(mode=ms.PYNATIVE_MODE)
net = Mul()
```
Such modifier usualy usefull when need to improve the execution speed of forward computing tasks in dynamic graph mode.


Another useful way of applying intermediate representation is to investigate errors of network definition. For example, lets consider an orinary shapes mismatch error. In Python native mode the output looks like below:

In [21]:
import numpy as np
import mindspore
from mindspore import nn, ops, set_context, Tensor, Parameter
from mindspore.common.initializer import initializer

ms.set_context(mode=ms.PYNATIVE_MODE)
ms.set_context(save_graphs=False)

class Net(nn.Cell):
    def __init__(self):
        super(Net, self).__init__()
        self.weight = Parameter(initializer('normal', [32, 8]), name="weight")
        self.bias = Parameter(initializer('zeros', [4]), name="bias")

        self.matmul = ops.MatMul()
        self.bias_add = ops.BiasAdd()

    def construct(self, x1):
        x = self.matmul(x1, self.weight)
        x = self.bias_add(x, self.bias)
        return x

net = Net()
x = Tensor(np.arange(3*32).reshape(3, 32), mindspore.float32)
out = net(x)
print('out', out.shape)

ValueError: ignored

So above the only way to understand the error is by using the ValueError text. Value error text doesn't look clear and we must appeal to output sizes (4 and 8) and stack trace in order to resolve the issue.

Now let us look at what does static graph mode when facing same error:

In [22]:
import numpy as np
import mindspore
from mindspore import nn, ops, set_context, Tensor, Parameter
from mindspore.common.initializer import initializer

ms.set_context(mode=ms.GRAPH_MODE)
ms.set_context(save_graphs=False)

class Net(nn.Cell):
    def __init__(self):
        super(Net, self).__init__()
        self.weight = Parameter(initializer('normal', [32, 8]), name="weight")
        self.bias = Parameter(initializer('zeros', [4]), name="bias")

        self.matmul = ops.MatMul()
        self.bias_add = ops.BiasAdd()

    def construct(self, x1):
        x = self.matmul(x1, self.weight)
        x = self.bias_add(x, self.bias)
        return x

net = Net()
x = Tensor(np.arange(3*32).reshape(3, 32), mindspore.float32)
out = net(x)
print('out', out.shape)

ValueError: ignored

We can find the generated file 'analyze_fail.dat':



In [ ]:

# 1.This file shows the parsed IR info when graph evaluating failed to help find the problem.
# 2.You can search the last `------------------------>` to the node which is inferred failed.
# 3.Refer to https://www.mindspore.cn/search?inputValue=analyze_fail.dat to get more instructions.
# ===============================================================================

# [No.1] Default_wrapper.284
# In file <ipython-input-22-3fdb641326b6>:18/
funcgraph fg_284(
        %para1 : Tensor(F32)[3, 32]    # x1
        , %para2 : Ref[Tensor(F32)][4]    # bias
        , %para3 : Ref[Tensor(F32)][32, 8]    # weight
    ) {

#------------------------> 0
    %1 = FuncGraph::fg_285(%para1)    #(Tensor(F32)[3, 32])    # fg_285=Default.285 #scope: Default
#[CNode]286
    Primitive::Return{prim_type=1}(%1)    #(Undefined) #scope: Default
      # In file <ipython-input-22-3fdb641326b6>:21/#[CNode]287
}
# order:
#   1: @Default_wrapper.284:[CNode]286{[0]: ValueNode<FuncGraph> Default.285, [1]: x1}
#   2: @Default_wrapper.284:[CNode]287{[0]: ValueNode<Primitive> Return, [1]: [CNode]286}


# [No.2] Default.285
# In file <ipython-input-22-3fdb641326b6>:18/
funcgraph fg_285[fg_284](
        %para4 : Tensor(F32)[3, 32]    # x1
    ) {
    %1 : Tensor(F32)[3, 8] = DoSignaturePrimitive::S-Prim-MatMul{prim_type=1}[output_names=["output"], transpose_a=Bool(0), input_names=["x1", "x2"], transpose_x2=Bool(0), transpose_x1=Bool(0), transpose_b=Bool(0)](%para4, %para3)    #(Tensor(F32)[3, 32], Ref[Tensor(F32)][32, 8]) #scope: Default
      # In file <ipython-input-22-3fdb641326b6>:19/#x

#------------------------> 1
    %2 = DoSignaturePrimitive::S-Prim-BiasAdd{prim_type=1}[output_names=["output"], format="NCHW", input_names=["x", "b"]](%1, %para2)    #(Tensor(F32)[3, 8], Ref[Tensor(F32)][4]) #scope: Default
      # In file <ipython-input-22-3fdb641326b6>:20/#x
    Primitive::Return{prim_type=1}(%2)    #(Undefined) #scope: Default
      # In file <ipython-input-22-3fdb641326b6>:21/#[CNode]288
}
# order:
#   1: @Default.285:x{[0]: ValueNode<DoSignaturePrimitive> S-Prim-MatMul, [1]: x1, [2]: weight}
#   2: @Default.285:x{[0]: ValueNode<DoSignaturePrimitive> S-Prim-BiasAdd, [1]: x, [2]: bias}
#   3: @Default.285:[CNode]288{[0]: ValueNode<Primitive> Return, [1]: x}


#===============================================================================
# num of function graphs in stack: 2/3 (Ignored 1 internal frames).

In short words the .dat file points to the exact place there error occurs and provides detailed information about the error. But it does require some [prerequisites](https://mindspore.cn/tutorials/experts/en/r1.10/debug/mindir.html#reading-ir) in order to read the IR files.


The last '--->' in the faile points to the position where inferring failed.

From that we can find the exact shapes mismatch: according to ...(%1, %para2)    #(Tensor(F32)[3, 8], Ref[Tensor(F32)][4]), BiasAdd’s inputs are %1 and %para2. That %1’ with shape [3, 8] and %para2 with shape [4] doesn’t meet the requirement about bias (Tensor) - The bias tensor, with shape (C). C must be the same as channel dimension C of input_x... for BiasAdd API. Thus, an error happens.

To resolve the error we need correct shape of the bias parameter:



```
...
self.bias = Parameter(initializer('zeros', [8]), name="bias")
...
```



Other useful features of IR files (.ir, .dat and .dot) is that they allow to track all the [phases](https://mindspore.cn/tutorials/experts/en/r1.10/debug/mindir.html#saving-ir) that were performed during computation graph compilation. Moreover, the .dot file might be [vizualized](https://mindspore.cn/tutorials/experts/en/r1.10/debug/mindir.html#dot-introduction) by certain tools.

